In [ ]:
from cell_paint_seg import utils, image_io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import exposure
import numpy as np
from skimage import measure

In [ ]:
channels = ["Brightfield", "ER", "AGP", "Mito", "DNA", "RNA"]

path_dir_im = "/Users/thomasathey/Documents/shavit-lab/fraenkel/first-sample/Assay Dev 20230329/BR00142688__2024-03-29T19_57_13-Measurement 1/deployment-test-small/tifs"
path_dir_seg_ilastik = "/Users/thomasathey/Documents/shavit-lab/fraenkel/first-sample/Assay Dev 20230329/BR00142688__2024-03-29T19_57_13-Measurement 1/deployment-test-small/segmentations"

tag_seg_cell = "ch7"
tag_seg_soma = "ch8"
tag_seg_nuc = "ch9"

reg_stat_limits = {"area": (-1, 4000)}

# View standard seg

In [ ]:
path_dir_seg_cp = "/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/cp-visualize/"
tag_cp = "r08"

# CellProfiler statistics which are used to filter segmented objects
path_cp_stats_nuc = "/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/cp-visualize/test_setNuclei.csv"
path_cp_stats_cell = "/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/cp-visualize/test_setCells.csv"

# Filtering options where key is the attribute (CellProfiler statistic name or regionprops property name) and value is tuple of lower and upper limits (inclusive)
cp_stat_limits = {"AreaShape_Area": (300, 4000), "Neighbors_NumberOfNeighbors_Adjacent": (-1, 5), "Neighbors_PercentTouching_Adjacent":(-1, 50)}
reg_stat_limits = {"area": (-1, 4000)}

# Collection of CellProfiler filtering options (for brevity)
kwargs = {"path_cp_stats_nuc":path_cp_stats_nuc, "path_cp_stats_cell":path_cp_stats_cell, "cp_stat_limits":cp_stat_limits}

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im, tag_cp)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_cp, tag_cp)


for id in id_to_path_im.keys():
    images = utils.read_ims(id_to_path_im[id])

    seg_nuc = utils.read_seg(id_to_path_seg_nuc[id])

    regions = utils.cp_filter(id, measure.regionprops(seg_nuc), **kwargs)
    seg_nuc_filtered = np.zeros_like(seg_nuc)
    for region in regions:
        seg_nuc_filtered[seg_nuc == region.label] = region.label



    image_dna = images[channels.index("DNA")]
    image_dna = exposure.equalize_adapthist(image_dna, clip_limit=0.03, kernel_size = [s // 64 for s in image_dna.shape])
    image_rna = images[channels.index("RNA")]
    image_rna = exposure.equalize_adapthist(image_rna, clip_limit=0.03)
    image_agp = images[channels.index("AGP")]
    image_agp = exposure.equalize_adapthist(image_agp, clip_limit=0.03)

    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)


    f, axs = plt.subplots(nrows=1, ncols=2, dpi=300)
    seg_nuc_masked = np.ma.masked_array(seg_nuc_filtered, mask=seg_nuc_filtered == 0)

    axs[0].imshow(image_rgb, cmap="gray")
    axs[0].set_title(f"Cell Painting of Cultured Neurons ({id}) \n AGP (red), RNA (green) and DNA (blue)", fontsize=24)

    # axs[1].imshow(image_dna, cmap="gray")
    # axs[1].imshow(seg_cell_masked, cmap="Set1", alpha=0.5)
    # axs[1].imshow(seg_soma_masked % 20, cmap="tab20", alpha=0.7)
    # axs[1].set_title("Cell Semantic Segmentation and Cell Body Instance Segmentation")

    axs[1].imshow(image_rgb, cmap="gray")
    axs[1].imshow(seg_nuc_masked % 20, cmap="tab20", alpha=0.7)
    axs[1].set_title("Cell (red) and Soma Cytoplasm (green) Semantic Segmentation \n Nucleus Instance Segmentation", fontsize=24)

    for ax in axs.flatten():
        ax.axis("off")
        
    f.set_size_inches(30, 10)
    f.tight_layout()
    #plt.show()
    #plt.savefig(f"/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/cp-visualize/cp_seg.svg")

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im, tag_cp)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_cp, tag_cp)


for id in id_to_path_im.keys():
    images = utils.read_ims(id_to_path_im[id])

    seg_nuc = utils.read_seg(id_to_path_seg_nuc[id])

    regions = utils.cp_filter(id, measure.regionprops(seg_nuc), **kwargs)
    seg_nuc_filtered = np.zeros_like(seg_nuc)
    for region in regions:
        seg_nuc_filtered[seg_nuc == region.label] = region.label



    image_dna = images[channels.index("DNA")]
    image_dna = exposure.equalize_adapthist(image_dna, clip_limit=0.03, kernel_size = [s // 64 for s in image_dna.shape])
    image_rna = images[channels.index("RNA")]
    image_rna = exposure.equalize_adapthist(image_rna, clip_limit=0.03)
    image_agp = images[channels.index("AGP")]
    image_agp = exposure.equalize_adapthist(image_agp, clip_limit=0.03)

    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)


    f, axs = plt.subplots(nrows=1, ncols=2, dpi=300)

    seg_nuc_filtered = seg_nuc_filtered > 0
    seg_nuc_masked = np.ma.masked_array(seg_nuc_filtered, mask=seg_nuc_filtered == 0)

    # image_rgb_wmask = image_rgb.copy()
    # max = np.amax(image_rgb)
    # for color in range(3):
    #     image_rgb_wmask_c = image_rgb_wmask[:,:,color]
    #     image_rgb_wmask_c[seg_nuc_filtered > 0] = max
    #     image_rgb_wmask[:,:,color] = image_rgb_wmask_c
    # image_rgb_wmask_filtered = np.ma.masked_array(image_rgb_wmask, mask=image_rgb_wmask == 0)

    axs[0].imshow(image_rgb, cmap="gray")
    axs[0].set_title(f"Cell Painting of Cultured Neurons ({id}) \n AGP (red), RNA (green) and DNA (blue)", fontsize=24)

    # axs[1].imshow(image_dna, cmap="gray")
    # axs[1].imshow(seg_cell_masked, cmap="Set1", alpha=0.5)
    # axs[1].imshow(seg_soma_masked % 20, cmap="tab20", alpha=0.7)
    # axs[1].set_title("Cell Semantic Segmentation and Cell Body Instance Segmentation")

    axs[1].imshow(image_rgb, cmap="gray")
    axs[1].imshow(seg_nuc_masked, cmap="gray", vmin=0, vmax = 1)
    axs[1].set_title("Cell (red) and Soma Cytoplasm (green) Semantic Segmentation \n Nucleus Instance Segmentation", fontsize=24)

    for ax in axs.flatten():
        ax.axis("off")
        
    f.set_size_inches(30, 10)
    f.tight_layout()
    #plt.show()
    plt.savefig(f"/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/cp-visualize/cp_seg_process.svg")

# View hierarchical seg

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im, tag=".tif")
id_to_path_seg_cell = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_cell)
id_to_path_seg_soma = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_soma)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_nuc)


for id in id_to_path_im.keys():
    images = image_io.read_ims(id_to_path_im[id])

    seg_cell = image_io.read_seg(id_to_path_seg_cell[id])
    seg_nuc = image_io.read_seg(id_to_path_seg_nuc[id])

    seg_soma = image_io.read_seg(id_to_path_seg_soma[id])
    
    seg_cyto = np.logical_and(seg_soma > 0, seg_nuc == 0)



    image_dna = images[channels.index("DNA")]
    image_dna = exposure.equalize_adapthist(image_dna, clip_limit=0.03, kernel_size = [s // 64 for s in image_dna.shape])
    image_rna = images[channels.index("RNA")]
    image_rna = exposure.equalize_adapthist(image_rna, clip_limit=0.03)
    image_agp = images[channels.index("AGP")]
    image_agp = exposure.equalize_adapthist(image_agp, clip_limit=0.03)

    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)

    f, axs = plt.subplots(nrows=1, ncols=2, dpi=300)

    seg_cell_masked = np.ma.masked_array(seg_cell, mask=seg_cell == 0)
    seg_soma_masked = np.ma.masked_array(seg_soma, mask=seg_soma == 0)
    seg_nuc_masked = np.ma.masked_array(seg_nuc, mask=seg_nuc == 0)
    seg_cyto_masked = np.ma.masked_array(seg_cyto, mask=seg_cyto == 0)

    axs[0].imshow(image_rgb, cmap="gray")
    axs[0].set_title(f"Image {id}", fontsize=24)

    axs[1].imshow(image_dna, cmap="gray")
    axs[1].imshow(seg_cell_masked % 20, cmap="tab20", alpha=0.5)
    axs[1].imshow(seg_cyto_masked, cmap="Accent", alpha=0.7)
    axs[1].imshow(seg_nuc_masked > 0, cmap="Set1", alpha=0.7)
    # axs[1].imshow(seg_cyto_masked, cmap="Accent", alpha=0.7)
    # axs[1].imshow(seg_nuc_masked > 0, cmap="Set1", alpha=0.7)
    axs[1].set_title(f"{len(regions)} Cells Detected", fontsize=24)
 
    for ax in axs.flatten():
        ax.axis("off")
        
    f.set_size_inches(30, 10)
    f.tight_layout()
    plt.show()
    #plt.savefig(f"/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/ex-stats/{id}.svg")

# Footprints

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im)
id_to_path_seg_cell = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_cell)
id_to_path_seg_soma = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_soma)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_nuc)


for id in id_to_path_im.keys():
    images = utils.read_ims(id_to_path_im[id])

    seg_cell = utils.read_seg(id_to_path_seg_cell[id])
    seg_nuc = utils.read_seg(id_to_path_seg_nuc[id])

    seg_soma = utils.read_seg(id_to_path_seg_soma[id])
    regions = utils.reg_prop_filter(measure.regionprops(seg_soma), reg_stat_limits)
    seg_soma_filtered = np.zeros_like(seg_soma)
    for region in regions:
        seg_soma_filtered[seg_soma == region.label] = region.label

    seg_nuc_filtered = np.copy(seg_soma_filtered)
    seg_nuc_filtered[seg_nuc == 0] = 0

    seg_cyto = np.logical_and(seg_soma_filtered > 0, seg_nuc_filtered == 0)



    image_dna = images[channels.index("DNA")]
    image_dna = exposure.equalize_adapthist(image_dna, clip_limit=0.03, kernel_size = [s // 64 for s in image_dna.shape])
    image_rna = images[channels.index("RNA")]
    image_rna = exposure.equalize_adapthist(image_rna, clip_limit=0.03)
    image_agp = images[channels.index("AGP")]
    image_agp = exposure.equalize_adapthist(image_agp, clip_limit=0.03)

    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)


    f, axs = plt.subplots(nrows=1, ncols=2, dpi=300)

    seg_cell_masked = np.ma.masked_array(seg_cell, mask=seg_cell == 0)
    seg_soma_masked = np.ma.masked_array(seg_soma_filtered, mask=seg_soma_filtered == 0)
    seg_nuc_masked = np.ma.masked_array(seg_nuc_filtered, mask=seg_nuc_filtered == 0)
    seg_cyto_masked = np.ma.masked_array(seg_cyto, mask=seg_cyto == 0)

    axs[0].imshow(seg_nuc_masked % 20, cmap="tab20")
    axs[0].set_title(f"Cell Bodies", fontsize=24)

    axs[1].imshow(seg_cell > 0, cmap="gray")
    axs[1].set_title(f"Cells with Processes", fontsize=24)
    #axs[1].set_title("Cell (red) and Soma Cytoplasm (green) Semantic Segmentation \n Nucleus Instance Segmentation", fontsize=24)

    for ax in axs.flatten():
        ax.axis("off")
        
    f.set_size_inches(30, 10)
    f.tight_layout()
    #plt.show()

# Stats

In [ ]:
id_to_path_im = utils.get_id_to_path(path_dir_im)
id_to_path_seg_cell = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_cell)
id_to_path_seg_soma = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_soma)
id_to_path_seg_nuc = utils.get_id_to_path(path_dir_seg_ilastik, tag_seg_nuc)

data = {}

for id in id_to_path_im.keys():
    images = utils.read_ims(id_to_path_im[id])

    seg_cell = utils.read_seg(id_to_path_seg_cell[id])
    seg_nuc = utils.read_seg(id_to_path_seg_nuc[id])

    seg_soma = utils.read_seg(id_to_path_seg_soma[id])
    regions = utils.reg_prop_filter(measure.regionprops(seg_soma), reg_stat_limits)
    seg_soma_filtered = np.zeros_like(seg_soma)
    for region in regions:
        seg_soma_filtered[seg_soma == region.label] = region.label

    seg_nuc_filtered = np.copy(seg_soma_filtered)
    seg_nuc_filtered[seg_nuc == 0] = 0

    seg_cyto = np.logical_and(seg_soma_filtered > 0, seg_nuc_filtered == 0)



    image_dna = images[channels.index("DNA")]
    image_rna = images[channels.index("RNA")]
    image_agp = images[channels.index("AGP")]

    image_rgb = np.stack([image_agp, image_rna, image_dna], axis=2)


    seg_cell_masked = np.ma.masked_array(seg_cell, mask=seg_cell == 0)
    seg_soma_masked = np.ma.masked_array(seg_soma_filtered, mask=seg_soma_filtered == 0)
    seg_nuc_masked = np.ma.masked_array(seg_nuc_filtered, mask=seg_nuc_filtered == 0)
    seg_cyto_masked = np.ma.masked_array(seg_cyto, mask=seg_cyto == 0)

    num_cells = len(np.unique(seg_nuc_filtered)) - 1
    av_cell_area = np.sum(seg_cell > 0) / num_cells
    areas = []
    for region in regions:
        areas.append(region.area)

    av_rna = np.sum(image_rna[seg_cell > 0]) / np.sum(seg_cell > 0)
    rnas = []
    for region in regions:
        rnas.append(np.sum(image_rna[seg_soma_filtered == region.label]) / np.sum(seg_soma_filtered == region.label))

    eccs = []
    for region in regions:
        eccs.append(region.eccentricity)


    data[id] = {"No. Cells": num_cells, "Av. Cell Area": av_cell_area, "Soma Areas": areas, "Av. RNA Content": av_rna, "RNA Content": rnas, "Soma Eccentricity": eccs}
    

In [ ]:
df_cell = pd.DataFrame({"Image ID": ["r16c07f02p01", "r05c06f09p01"],
                        "Number of Cells": [data[id]["No. Cells"] for id in ["r16c07f02p01", "r05c06f09p01"]],
                        "Average Cell Area": [data[id]["Av. Cell Area"] for id in ["r16c07f02p01", "r05c06f09p01"]],
                        "Average RNA Content": [data[id]["Av. RNA Content"] for id in ["r16c07f02p01", "r05c06f09p01"]]})
f, axs = plt.subplots(ncols=2)
sns.barplot(df_cell, x="Image ID", y="Average Cell Area", ax=axs[0])
sns.barplot(df_cell, x="Image ID", y="Average RNA Content", ax=axs[1])
plt.suptitle("Cell Statistics (Semantic Segmentation)")
f.tight_layout()
plt.savefig("/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/ex-stats/semantic.svg")
        

In [ ]:
data_ids = []
data_areas = []
data_rna = []
data_ecc = []

for id in ["r16c07f02p01", "r05c06f09p01"]:
    for area, rna, ecc in zip(data[id]["Soma Areas"], data[id]["RNA Content"], data[id]["Soma Eccentricity"]):
        data_ids.append(id)
        data_areas.append(area)
        data_rna.append(rna)
        data_ecc.append(ecc)



df_soma = pd.DataFrame({"Image ID": data_ids,
                        "Soma Area": data_areas,
                        "Soma RNA Content": data_rna,
                        "Soma Eccentricity": data_ecc})
f, axs = plt.subplots(ncols=3)
sns.histplot(df_soma, stat="probability", common_norm=False, x="Soma Area", hue="Image ID", ax=axs[0])
sns.histplot(df_soma, stat="probability", common_norm=False, x="Soma RNA Content", hue="Image ID", ax=axs[1])
sns.histplot(df_soma, stat="probability", common_norm=False, x="Soma Eccentricity", hue="Image ID", ax=axs[2])
f.set_figwidth(8)
plt.suptitle("Soma Statistics (Instance Segmentation)")
f.tight_layout()
plt.savefig("/Users/thomasathey/Documents/shavit-lab/fraenkel/presentation/hierarchical/ex-stats/instance.svg")